In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import gym
from amalearn.environment import EnvironmentBase
A = 0
B = 1

### Parameters given in the question

In [ ]:
class Parameters:
    def __init__(self):
        self.max_capacity = 20
        self.max_purchase = 5
        self.data_cost = 10
        self.purchase_cost = 2 #for part b, change it to 6
        self.theta = 0.01
        self.gamma = 0.9 #for part a.2, change to 1 
        self.lambda_request_A = 3
        self.lambda_request_B = 4
        self.lambda_terminate_A = 3
        self.lambda_terminate_B = 2
        self.upper_bound = 3

### Poisson Distribution for caclulating probability of a going to a state

In [ ]:
def poisson(lambdaa, occurrences_count) : return (lambdaa**occurrences_count) * np.exp(-lambdaa) / math.factorial(occurrences_count)

### The main class for solving the problem using PI

In [ ]:
import numpy as np

class PolicyIteration(object):
    def __init__(self, params, env):
      self.env = env
      self.params = params
      self.states = [(x, y) for x in range(self.params.max_capacity + 1) for y in range(self.params.max_capacity + 1)]
      self.policy = np.zeros([self.params.max_capacity + 1]*2, int)
      self.policies = []

    def policy_eval(self, policy):
      while True:
        delta = 0
        for state in self.states:
            v = env.state_value[state]
            env.set_current_state(state)
            env.state_value[state] = env.calculate_reward(self.policy[state])
            delta = max(delta, abs(env.state_value[state] - v))
        if delta < self.params.theta: break
        print("Delta is:", delta)


    def solve_problem(self):
      iter = 0
      while True:
        print("iteration #",iter)
        iter +=1
        self.policies.append(self.policy.copy())
        self.policy_eval(self.policy)
        policy_stable = True
        for state in self.states:
            old_action = self.policy[state]
            env.set_current_state(state)
            values_dict = {action: env.calculate_reward(action) for action in env.available_actions()}
            self.policy[state] = np.random.choice([action for action, value in values_dict.items() if value == max(list(values_dict.values()))])
            if old_action != self.policy[state]: policy_stable = False
        if policy_stable: break

    def print_policies(self):
        for idx, policy in enumerate(self.policies):
          plt.figure()
          plt.imshow(policy, origin='lower', interpolation='none', vmin=-self.params.max_purchase, vmax=self.params.max_purchase, cmap = 'Purples')
          plt.xlabel('#Data at second company')
          plt.ylabel('#Data at first company')
          plt.title('policy{:d}'.format(idx))
          plt.colorbar()

    def print_value(self):
        fig = plt.figure()
        fig.set_size_inches(8, 5)
        ax = fig.gca(projection='3d')
        X = np.arange(0, self.params.max_capacity + 1)
        Y = np.arange(0, self.params.max_capacity + 1)
        X, Y = np.meshgrid(X, Y)
        ax.plot_surface(X, Y, env.state_value, cmap = 'Purples')


### Environment for our agent 

In [ ]:
class MDPEnvironment_(EnvironmentBase):
    def __init__(self, params):

        self.current_state = None
        self.params = params
        n_states = (self.params.max_capacity + 1)**2
        n_actions = 11
        self.state_value = np.zeros([self.params.max_capacity + 1]*2)
        state_space = gym.spaces.Discrete(n_states)
        action_space = gym.spaces.Discrete(n_actions)
        self.reset()
        super().__init__(action_space, state_space, id, None)

    def next_state(self, action):
        self.episode_length += 1
        ns = np.random.choice(list(range(self.observation_space.n)), p=self.P_s_s_a[self.s_current, :, action])
        return ns

    def available_actions(self):
        return np.arange(self.action_space.n, dtype='int32')

    def terminated(self): return self.episode_length > 20

    def observe(self): return self.s_current

    def get_info(self, action): return {}

    def reset(self):
        self.s_current = 0
        self.s_previous = None
        self.episode_length = 0

    def close(self): return

    def render(self, mode='human'): return

    def set_current_state(self, state): self.current_state = state

    def available_actions(self):
        actions = []
        given_state = self.current_state
        A_state, B_state = given_state
        all_actions = [act for act in range(-self.params.max_purchase, self.params.max_purchase + 1)]
        for action in all_actions:
            if A_state - action < 0 or A_state - action > self.params.max_capacity: continue 
            if B_state + action < 0 or B_state + action > self.params.max_capacity: continue
            actions.append(action)
        print("Available actions:", actions, "State:",given_state )
        return actions


    def calculate_reward(self, action):
        states = self.current_state
        returns = 0
        returns -= self.params.purchase_cost * abs(action)
        data_A = int(min(states[A] - action, self.params.max_capacity))
        data_B = int(min(states[B] + action, self.params.max_capacity))
        for request_A in range(self.params.upper_bound):
            for request_B in range(self.params.upper_bound):
                requests_probability = poisson(self.params.lambda_request_A, request_A) * poisson(self.params.lambda_request_B,request_B)
                total_requestA, total_requestB = min(data_A, request_A), min(data_B, request_B)
                rewards = (total_requestA + total_requestB) * self.params.data_cost
                for terminate_A in range(self.params.upper_bound):
                    for terminate_B in range(self.params.upper_bound):
                        returns_probability = poisson(self.params.lambda_terminate_A, terminate_A) * poisson(self.params.lambda_terminate_B, terminate_B)
                        left_data_A = min(data_A - total_requestA + terminate_A, self.params.max_capacity)
                        left_data_B = min(data_B - total_requestB + terminate_B, self.params.max_capacity)
                        returns += returns_probability * requests_probability * (rewards + self.params.gamma * self.state_value[left_data_A, left_data_B])
        return returns



In [ ]:

params = Parameters()
env = MDPEnvironment_(params)
policy_iteration = PolicyIteration(params, env)
policy_iteration.solve_problem()


In [ ]:
policy_iteration.print_policies()


In [ ]:
policy_iteration.print_value()